# New plotting examples

In [ ]:
%matplotlib widget
import numpy as np
import scipp as sc
from scipp.plt import *
from scipp.plt.masks import *

## Helper functions to generate data

In [ ]:
def make_1d_data():
    N = 50
    x = sc.arange('tof', float(N), unit='us')
    a = sc.DataArray(data=sc.array(dims=['tof'], values=5.0 + 5.0*np.sin(np.arange(float(N))/5.0),
                                    variances=np.random.random(N),
                                    unit='counts'),
                      coords={'tof': x}, name='a')
    b = sc.DataArray(data=sc.array(dims=['tof'], values=np.random.random(N),
                                    variances=np.random.random(N),
                                    unit='counts'),
                      coords={'tof': x}, name='b')
    return {'a': a, 'b': b}

def make_2d_data():
    N = 300
    M = 150
    xx = np.arange(N, dtype=np.float64)
    yy = np.arange(M, dtype=np.float64)
    x, y = np.meshgrid(xx, yy)
    b = N/20.0
    c = M/2.0
    r = np.sqrt(((x-c)/b)**2 + (y/b)**2)
    a = np.sin(r)
    return sc.DataArray(
        data=sc.array(dims=['y', 'x'], values=a, unit='K'),
        coords={'x': sc.array(dims=['x'], values=xx, unit='m'),
                'y': sc.array(dims=['y'], values=yy, unit='m')})

def make_3d_data():
    N = 20
    M = 30
    L = 20
    xx = np.arange(N, dtype=np.float64)
    yy = np.arange(M, dtype=np.float64)
    zz = np.arange(L, dtype=np.float64)
    x, y, z = np.meshgrid(xx, yy, zz, indexing='ij')
    b = N/20.0
    c = M/2.0
    d = L/2.0
    r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2 + ((z-d)/b)**2)
    a = np.sin(r)
    return sc.DataArray(data=sc.array(dims=['x', 'y', 'z'], values=a),
                      coords = {'x': sc.array(dims=['x'], values=xx),
                                'y': sc.array(dims=['y'], values=yy),
                                'z': sc.array(dims=['z'], values=zz)})

## 1D line plot

In [ ]:
da = make_1d_data()['a']

In [ ]:
m = Model(da)
fig = Figure()
m.add_view("root", fig)
p = Plot()
p.add_model("da", m)
p

## Two 1D lines

In [ ]:
data = make_1d_data()

In [ ]:
m_a = Model(data['a'])
m_b = Model(data['b'])
fig = Figure()
m_a.add_view("root", fig)
m_b.add_view("root", fig)
p = Plot()
p.add_model("a", m_a)
p.add_model("b", m_b)
p

## 2D image

In [ ]:
da = make_2d_data()

In [ ]:
m = Model(da)
fig = Figure()
m.add_view("root", fig)
p = Plot()
p.add_model("da", m)
p

## 2D image with gaussian smoothing slider

In [ ]:
da = make_2d_data()

In [ ]:
m = Model(da)

In [ ]:
from scipy.ndimage import gaussian_filter
import ipywidgets as ipw

def smooth(da, sigma):
    out = da.copy()
    smoothed = sc.array(
        dims=da.dims,
        values=gaussian_filter(da.values, sigma=sigma),
        unit=da.unit)
    out.data = smoothed
    return out

sl = ipw.IntSlider(min=1, max=50)

smooth_node = Node(func=smooth)
smooth_view = WidgetView(widgets={"sigma": sl})

In [ ]:
m.insert("smooth", smooth_node, after=m.end().name)
m.show()

In [ ]:
f = Figure()
m.add_view("smooth", f)
m.add_view("smooth", smooth_view)

In [ ]:
smooth_node.func

In [ ]:
p = Plot()
p.add_model("da", m)
p

## 2D image with masks

In [ ]:
da = make_2d_data()
da.masks['m1'] = da.data < sc.scalar(0.0, unit='K')
da.masks['m2'] = da.coords['x'] > sc.scalar(80., unit='m')

In [ ]:
masks_node = Node(func=hide_masks)
masks_view = WidgetView(widgets={"masks": MaskWidget(masks=da.masks)})
m = Model(da)
m.insert("hiding_masks", masks_node, after=m.end().name)

In [ ]:
f = Figure()
m.add_view("hiding_masks", f)
m.add_view("hiding_masks", masks_view)
m.show()

In [ ]:
p = Plot()
p.add_model("da", m)
p

## Two 1D lines with masks

In [ ]:
data = make_1d_data()
data['a'].masks['m1'] = data['a'].coords['tof'] > sc.scalar(40.0, unit='us')
data['a'].masks['m2'] = data['a'].data < data['b'].data
data['b'].masks['m1'] = data['b'].coords['tof'] < sc.scalar(5.0, unit='us')

In [ ]:
m_a = Model(data['a'])
m_b = Model(data['b'])

a_masks_node = Node(func=hide_masks)
a_masks_view = WidgetView(widgets={"masks": MaskWidget(masks=data['a'].masks)})

b_masks_node = Node(func=hide_masks)
b_masks_view = WidgetView(widgets={"masks": MaskWidget(masks=data['b'].masks)})

m_a.insert("hiding_masks_for_a", a_masks_node, after=m_a.end().name)
m_b.insert("hiding_masks_for_b", b_masks_node, after=m_b.end().name)

m_a.add_view("hiding_masks_for_a", a_masks_view)
m_b.add_view("hiding_masks_for_b", b_masks_view)

fig = Figure()
m_a.add_view("hiding_masks_for_a", fig)
m_b.add_view("hiding_masks_for_b", fig)

p = Plot()
p.add_model("a", m_a)
p.add_model("b", m_b)
p

## Node that sums the data along one dimension

In [ ]:
da = make_2d_data()

def squash(da):
    return da.sum('y')

m = Model(da)
m.insert("sum_y", Node(func=squash), after=m.end().name)

f1d = Figure()
f2d = Figure()
m.add_view("root", f2d)
m.add_view("sum_y", f1d)

p = Plot()
p.add_model("da", m)
p

In [ ]:
m.show()

## Slicing 3D cube into 2D image with slider

In [ ]:
import numpy as np
import scipp as sc
from scipp.plt import plot

da = make_3d_data()

from scipp.plt.slicing_filter import SlicingFilter
sl = SlicingFilter(dims=da.dims,
                                sizes=da.sizes,
                                ndim=2)
plot(da, filters=[sl])

## Slicing 3D cube with histograms on sides of both axes

In [ ]:
def hist(da):
    return da.sum('y')

hist_node = Node(name="hist", func=hist)
m.insert("hist", hist_node, after=m.end().name)
h = Figure()
m.add_view("hist", h)
m.show()

In [ ]:
h